In [1]:
!pwd

/home/mhossai5/Ag-Ab-Affinity2/LLM+Graph


In [2]:
import os
import re
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# ProtT5 imports
from transformers import T5Tokenizer, T5EncoderModel
# AbLang2
import ablang2

tqdm.pandas()

2025-12-04 14:46:03.071045: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 14:46:03.698462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-04 14:46:03.698504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-04 14:46:03.710629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-04 14:46:03.767647: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

[2025-12-04 14:46:41,938] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: warning: libstdc++.so.6, needed by /home/mhossai5/local/cuda-11.8/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::string::substr(unsigned long, unsigned long) const@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.

In [3]:
# ============================================================
# DEVICE SETUP
# ============================================================

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"\n[INFO] Using device: {device}\n")


[INFO] Using device: cuda:0



In [4]:
# ============================================================
# LOAD ABLANG2 (Antibody Model)
# ============================================================

def load_ablang2():
    model_name = "ablang2-paired"
    print(f"[INFO] Loading AbLang2 model: {model_name}")

    model = ablang2.pretrained(
        model_to_use=model_name,
        random_init=False,
        device=device
    )
    return model

ab_model = load_ablang2()

# ============================================================
# LOAD Encoder (Antigen Model)
# ============================================================

def load_ag():
    model_name = "Rostlab/prot_t5_xl_half_uniref50-enc"
    print(f"[INFO] Loading ProtT5 model: {model_name}")

    tokenizer = T5Tokenizer.from_pretrained(
        model_name,
        do_lower_case=False
    )
    model = T5EncoderModel.from_pretrained(model_name)

    # FP16 for GPU, FP32 for CPU
    if device.type == "cuda":
        model = model.half()
    else:
        model = model.float()

    model = model.to(device).eval()
    return model, tokenizer

ag_model, ag_tokenizer = load_ag()

[INFO] Loading AbLang2 model: ablang2-paired


/home/mhossai5/.conda/envs/llm_tor2/lib/python3.10/site-packages/ablang2/load_model.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


[INFO] Loading ProtT5 model: Rostlab/prot_t5_xl_half_uniref50-enc


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# ============================================================
# DATA LOADING & CLEANING
# ============================================================

DATA_PATH = '../RLEAAI/data/HIV/'
os.makedirs(os.path.dirname(DATA_PATH),exist_ok =True)
df = pd.read_excel("https://github.com/zhouyu9931/RLEAAI/raw/refs/heads/main/data/dataset_hiv.xlsx")
df.head()

,antibody_seq,virus_seq,label,split
0,QMKLMQSGGVMVRPGESATLSCVASGFDFSRNGFEWLRQGPGKGLQ...,MRVMGIRKNYQHLWREGILLLGILMICSAADNLWVTVYYGVPVWRE...,0,seen
1,QPQLQESGPGLVEASETLSLTCTVSGDSTGRCNYFWGWVRQPPGKG...,MRVRGIPRNWPQWWIWGILGFWMIIICRVVGNMWVTVYYGVPVWTD...,0,seen
2,QVQLLQSGAAVTKPGASVRVSCEASGYNIRDYFIHWWRQAPGQGLQ...,MRVMEIQRNCQHWWIWGILGFWMLMICNVRGWWVTVYYGVPVWKEA...,1,seen
3,QSQLQESGPRLVEASETLSLTCNVSGESTGACTYFWGWVRQAPGKG...,MRVKETQMNWPNLWKLGTLILGLVIICSASXNLWVTVYYGVPVWRD...,1,seen
4,QEQLVESGGGVVQPGGSLRLSCLASGFTFHKYGMHWVRQAPGKGLE...,MRVTGTQRNCQQWWIWIWIILGFWWMLMMCKGEKLWVTVYYGVPVW...,1,seen


In [6]:
df['split'].value_counts()

split
seen      24843
unseen     4551
Name: count, dtype: int64

In [7]:
# ============================================================
# EMBEDDING FUNCTIONS
# ============================================================

def embed_antibody(hseq: str, lseq: str):
    """
    Returns AbLang2 embeddings for (heavy, light) sequences.
    """
    seq = f"{hseq}|{lseq}".upper() # VH first, VL second, with | used to separated the two sequences 
    tokenized_seq = ab_model.tokenizer([seq], pad=True, w_extra_tkns=False, device=device)

    # Generate rescodings
    with torch.no_grad():
        rescoding = ab_model.AbRep(tokenized_seq).last_hidden_states
    return rescoding.squeeze().detach().cpu().numpy()#.reshape(-1)


# def embed_antigen(seq: str):
#     """
#     Returns Encoder embeddings for antigen. shape 2560
#     """
#     tokens = ag_tokenizer(
#         seq,
#         return_tensors="pt",
#         add_special_tokens=True
#     )

#     tokens = {k: v.to(device) for k, v in tokens.items()}

#     with torch.no_grad():
#         output = ag_model(**tokens)

#     return output.last_hidden_state.squeeze(0).cpu().numpy() #.reshape(-1)
def embed_antigen(seq: str):
    seq = " ".join(list(re.sub(r"[UZOB]", "X", seq)))
    ids = ag_tokenizer.encode_plus(seq, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(ids['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = ag_model(input_ids=input_ids, attention_mask=attention_mask)
    token_embeddings = embedding.last_hidden_state.squeeze(0) 
    mask = attention_mask.squeeze(0).bool()  
    filtered_embeddings = token_embeddings[mask]
    return filtered_embeddings.squeeze().detach().cpu().numpy()

In [11]:
import h5py
import numpy as np
from tqdm import tqdm
file_path = f"{DATA_PATH}/HIV-RLEAAI-ProtT5-Full.h5"
seq_list = pd.unique(df[['antibody_seq','virus_seq']].values.ravel())
with h5py.File(file_path, 'w') as hf:
    for seq in tqdm(seq_list):
        emb = embed_antigen(seq).reshape(-1)
        hf.create_dataset(seq, data=emb, compression="gzip")
print(f"\n[INFO] Embeddings will be saved to:\n{file_path}\n")

100%|██████████| 1194/1194 [01:38<00:00, 12.06it/s]



[INFO] Embeddings will be saved to:
../RLEAAI/data/HIV//HIV-RLEAAI-ProtT5-Full.h5



In [12]:
df.iloc[:1].progress_apply(lambda x: embed_antigen(x['antibody_seq']).shape, axis= 1)

  0%|          | 0/1 [00:00<?, ?it/s]

0    (244, 1024)
dtype: object

In [3]:
!ls data/test-case-data

1HZH.pdb  5TE4.pdb	       rcsb_pdb_1RZ7.fasta  rcsb_pdb_6PEH.fasta
1RZ7.pdb  6PEH.pdb	       rcsb_pdb_3MOA.fasta
3MOA.pdb  rcsb_pdb_1HZH.fasta  rcsb_pdb_5TE4.fasta


In [24]:
fasta_df = pd.read_csv("./data/test-case-data/file.csv")
fasta_df['ab_seq'] = fasta_df['light_antibody']+fasta_df['heavy_antibody']
display(fasta_df)

,antibody_chain_id,light_antibody,heavy_antibody,antigen_chain_id,antigen_sequence,ab_seq
0,6NFC,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,EVQLVETGGGLVQPGGSLKLSCRASGYTFSSFAMSWVRQAPGKGLE...,6NFC_4|Chains,AVGIGAVSLGFLGAAGSTMGAASMTLTVQARNLLSGIVQQQSNLLR...,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...
1,6NFC,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,EVQLVETGGGLVQPGGSLKLSCRASGYTFSSFAMSWVRQAPGKGLE...,6NFC_3|Chains,AENLWVTVYYGVPVWKDAETTLFCASDAKAYETEKHNVWATHACVP...,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...
2,3MOA,ALQLTQSPSSLSASVGDRITITCRASQGVTSALAWYRQKPGSPPQL...,RITLKESGPPLVKPTQTLTLTCSFSGFSLSDFGVGVGWIRQPPGKA...,3MOA_1,EKNEQELLELDKWASLWX,ALQLTQSPSSLSASVGDRITITCRASQGVTSALAWYRQKPGSPPQL...
3,5TE4,YIHVTQSPSSLSVSIGDRVTINCQTSQGVGSDLHWYQHKPGRAPKL...,RAHLVQSGTAMKKPGASVRVSCQTSGYTFTAHILFWFRQAPGRGLE...,5TE4_1,AWEDADTTLFCASDAKAYSTEKHNVWATHACVPTDPDPQEIPLENV...,YIHVTQSPSSLSVSIGDRVTINCQTSQGVGSDLHWYQHKPGRAPKL...
4,5DD0,DIQVTQSPSSLSASVGDTVTISCRTSQSISTWLAWYQVKPGKAPKL...,QVQLQESGPGLVKPSETLSVTCAVSGVSFSSFWWGWIRQSPGKGLE...,5DD0_3,XLLELDKWASLWX,DIQVTQSPSSLSASVGDTVTISCRTSQSISTWLAWYQVKPGKAPKL...


In [25]:
df = pd.read_excel("https://github.com/zhouyu9931/RLEAAI/raw/refs/heads/main/data/dataset_hiv.xlsx")
unseen_df = df[df['split']=='unseen'].copy()
unseen_df

,antibody_seq,virus_seq,label,split
15,QVQLVQSGGQMKKPGESMRISCRASGYEFIDCTLNWIRLAPGKRPE...,MRAKEMRKSCQHLWKWGILLFGMLMICSAEEKLWVTVYYGVPVWKE...,1,unseen
19,QGQLVQSGAELKKPGASVKISCKTSGYKFSFFHINWIRQTAGRGPE...,MRVRGIPRNWPQWWIWGILGFWMIIICRVVGNMWVTVYYGVPVWTD...,0,unseen
21,QVQLVQSGAEVKKPGESLKISCKVSGYNFASEWIGWVRQMPGKGLE...,MKVMGIQKNYPSFWRWGMILFWIMMICNATNLWVTVYYGVPVWRDA...,0,unseen
31,QVQLVQSGAEVKKPGSSVKVSCKASGGTFRSYAISWVRQAPGQGLE...,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVYYGVPVWKE...,1,unseen
32,QVQLVQSGAEMKMPGASVKVSCKASGYTFTGNYIHWVRQAPGQGLE...,MRVMGILRSYQQWWIWGILGFWMLMICNVWGNLWVTVYYGVPVWKE...,0,unseen
...,...,...,...,...
29346,QGQLVQSGGGLKKPGTSVTISCLASEYTFNEFVIHWIRQAPGQGPL...,MRVTEIRKSYQHWWRWGIMLLGMLMICNAEEKLWVTVYYGVPVWKE...,1,unseen
29347,QLQLQESGPGLVKPSETLSLTCTVSGGSISDFNYYWGWIRQPPGKG...,MRVMGIQRNSQCFLSWGMLVLGIMMICSAVGNLWVTVYYGVPVWKD...,0,unseen
29364,QVQLVQSGGGLVKPGGSLTLSCSASGFFFDNSWMGWVRQAPGKGLE...,MRVRGIQRNWPQWWIWGILGLLMIIICRGVGNMWVTVYYGVPVWTE...,1,unseen
29379,EVQLVESGGGLIRPGGSLRLSCKGSGFIFENFGFGWVRQGPGKGLE...,MRVRGIQTSWQNLWRWGTMILGMLMIYSAAENLWVTVYYGVPVWKD...,0,unseen


In [28]:
fasta_df['ab_seq'].isin(df['antibody_seq']),fasta_df['antigen_sequence'].isin(df['virus_seq'])

(0    False
 1    False
 2    False
 3    False
 4    False
 Name: ab_seq, dtype: bool,
 0    False
 1    False
 2    False
 3    False
 4    False
 Name: antigen_sequence, dtype: bool)

In [35]:
fasta_df = pd.concat([
df[df['split']=='unseen'].groupby(['label'],group_keys =False).sample(2).rename(columns ={'antibody_seq':'ab_seq','virus_seq':'antigen_sequence'})[['ab_seq','antigen_sequence']],
    fasta_df,],axis= 0)
fasta_df

,ab_seq,antigen_sequence,antibody_chain_id,light_antibody,heavy_antibody,antigen_chain_id,ab_emb,at_emb
4942,QLQLQESGPGLVKPSETLSLTCTVSGGSISDFNYYWGWIRQPPGKG...,MKVKGTRTNWQHLWKWGTMLLGMLMICSVAGNWVTVYYGVPVWKEA...,NaN,NaN,NaN,NaN,NaN,NaN
7229,QGQLVQSGGELKKPGASVKISCKTSGYRFSFYHINWIRQVSGRGPE...,MRVKGIKKSFQHWWKWGTMLLGILMICSATDKLWVTVYYGVPVWKE...,NaN,NaN,NaN,NaN,NaN,NaN
27921,EVQLVESGGGLVKPGGSLRLSCAASGFDFDNAWMTWVRQPPGKGLE...,MRVMGIQRNCQHLLTWGIMILGTIIFCSAVENLWVTVYYGVPVWRD...,NaN,NaN,NaN,NaN,NaN,NaN
5971,QVRLSQSGGQMKKPGESMRLSCRASGYEFLNCPINWIRLAPGRRPE...,MRVMGIQRNCQHLLTWGIMILGTIIFCSAVENLWVTVYYGVPVWRD...,NaN,NaN,NaN,NaN,NaN,NaN
0,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,AVGIGAVSLGFLGAAGSTMGAASMTLTVQARNLLSGIVQQQSNLLR...,6NFC,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,EVQLVETGGGLVQPGGSLKLSCRASGYTFSSFAMSWVRQAPGKGLE...,6NFC_4|Chains,"[0.03604, -0.1057, -0.2122, 0.0449, -0.2957, -...","[0.4302, -0.2279, -0.3447, 0.08966, -0.3872, -..."
1,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,AENLWVTVYYGVPVWKDAETTLFCASDAKAYETEKHNVWATHACVP...,6NFC,ALTQPPSVSGSPGQSVTISCTGTSSDIGSYNYVSWYQQHPGKAPKL...,EVQLVETGGGLVQPGGSLKLSCRASGYTFSSFAMSWVRQAPGKGLE...,6NFC_3|Chains,"[0.03604, -0.1057, -0.2122, 0.0449, -0.2957, -...","[0.1974, -0.3628, -0.1719, 0.3147, -0.752, -0...."
2,ALQLTQSPSSLSASVGDRITITCRASQGVTSALAWYRQKPGSPPQL...,EKNEQELLELDKWASLWX,3MOA,ALQLTQSPSSLSASVGDRITITCRASQGVTSALAWYRQKPGSPPQL...,RITLKESGPPLVKPTQTLTLTCSFSGFSLSDFGVGVGWIRQPPGKA...,3MOA_1,"[0.0454, -0.3665, -0.1304, 0.2457, -0.4275, -0...","[0.05383, -0.353, -0.08923, -0.1849, 0.011604,..."
3,YIHVTQSPSSLSVSIGDRVTINCQTSQGVGSDLHWYQHKPGRAPKL...,AWEDADTTLFCASDAKAYSTEKHNVWATHACVPTDPDPQEIPLENV...,5TE4,YIHVTQSPSSLSVSIGDRVTINCQTSQGVGSDLHWYQHKPGRAPKL...,RAHLVQSGTAMKKPGASVRVSCQTSGYTFTAHILFWFRQAPGRGLE...,5TE4_1,"[-0.144, -0.4844, -0.03342, 0.1979, -0.388, -0...","[0.11597, -0.284, 0.11523, 0.1272, -0.5576, -0..."
4,DIQVTQSPSSLSASVGDTVTISCRTSQSISTWLAWYQVKPGKAPKL...,XLLELDKWASLWX,5DD0,DIQVTQSPSSLSASVGDTVTISCRTSQSISTWLAWYQVKPGKAPKL...,QVQLQESGPGLVKPSETLSVTCAVSGVSFSSFWWGWIRQSPGKGLE...,5DD0_3,"[-0.135, -0.3188, -0.03745, 0.3435, -0.2666, -...","[0.3613, 0.02461, -0.1746, -0.3618, -0.6055, 0..."


In [36]:
fasta_df['ab_emb'] = fasta_df.progress_apply(lambda x: embed_antigen(x['ab_seq']).flatten(), axis= 1)
fasta_df['at_emb'] = fasta_df.progress_apply(lambda x: embed_antigen(x['antigen_sequence']).flatten(), axis= 1)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [37]:
file_path = f"{DATA_PATH}/HIV-RLEAAI-ProtT5-Full.h5"
fasta_df.to_parquet(f"{DATA_PATH}/HIV-RLEAAI-ProtT5-Full.parquet",index =False)